In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
#import statsmodels.api as sm
#import itertools as it
#from statsmodels.sandbox.stats.multicomp import multipletests
#from itertools import compress
from pyBedGraph import BedGraph
from pybedtools import BedTool
import scipy.stats
from collections import Counter

In [12]:
def read_loadingfile(directory, file_name):
    with open(directory + file_name) as f:
        crnpk = {}
        #next(f)
        for line in f:
            tmp = line.strip().split("\t")[:-1]
            region = tmp[0]+":"+tmp[1]+"-"+tmp[2] + ";" + tmp[6]
            tmp[1] = int(tmp[1]) # peak start
            tmp[2] = int(tmp[2]) # peak end
            tmp[4] = int(tmp[4]) # gene start
            tmp[5] = int(tmp[5]) # gene end
            tmp[10] = float(tmp[10]) # TPM
            tmp[11] = float(tmp[11]) # RPKM
            strand = tmp[6]
            if strand == '+': # positive strand
                dist = min(abs(tmp[4]-tmp[2]), abs(tmp[4]-tmp[1]))
            else: # negative strand
                dist = min(abs(tmp[5]-tmp[2]), abs(tmp[5]-tmp[1]))
            tmp.append(dist) #distance to promoter;  tmp[12]
            tmp.append(tmp[5]-tmp[4]) # gene length; tmp[13]
            if tmp[12] < 5000 and tmp[10] > 0.5 and tmp[13] > 5000: # distance to promoter < 5kb and TMP > 0.5 & length > 5kb
                if region not in crnpk.keys():
                    crnpk[region] = [tmp]
                else:
                    crnpk[region].append(tmp)
            else:
                tmp = [tmp[0], tmp[1], tmp[2], '.', 0, 0, '.', '.', '.', '.', 0, 0, 0, 0]
                if region not in crnpk.keys():
                    crnpk[region] = [tmp]
                else:
                    if tmp not in crnpk[region]:
                        crnpk[region].append(tmp)
    return crnpk

In [13]:
def write_result(directory, out_list, out_name):
    with open(directory+out_name, 'a') as file1:
        for i in range(len(out_list)):
            file1.write('\t'.join(map(str, out_list[i])) + '\n')
    file1.close()

In [14]:
directory = '/Users/kimm/Desktop/GM12878_files/'
#gtf_file='hg38.ensGene.gtf'
#gtf_file = 'Homo_sapiens.GRCh38.100.gtf'
anchor_file = 'RNAPII-peaks-overlap_CTCF_motif_cohesin_annot_ENCFF879FKF_20200711.bed'
loading_file = 'RNAPII-peaks-overlap_NIPBL_cohesin-notoverlap_CTCF_motif_annot_ENCFF879FKF_20200711.bed'
#comp_file='Cohesin_0.2Pass_List1.txt'
#comp_file='Cohesin_All_20200512_List1.txt'

In [15]:
loading = read_loadingfile(directory, loading_file)

In [16]:
len(Counter([x.split(";")[0] for x in loading.keys()]).keys())

3638

In [17]:
filtered = {}
for key, val in loading.items():
    #print(key)
    #print(val)
    #maxtpm = val[0][11]
    #maxlength = val[0][14]
    #geneid = val[0][9]
    final = val[0]
    for x in val:
        if x[10] > 1.5*final[10]: ## tpm
            if x[8] == final[8]: ## gene id same
                if x[13] > final[13]: # max length of gene 
                    final = x
                else: 
                    final[10] = x[10]
            else: 
                final = x
    #print("final is: ")
    #print(final)
    #print("\n")
    #bed = [final[0], ]
    if key.split(';')[0] not in filtered.keys():
        filtered[key.split(';')[0]] = [[],[]]
    if x[6] == '+':
        filtered[key.split(';')[0]][0] = final
    elif x[6] == '-':
        filtered[key.split(';')[0]][1] = final
    #crnpk[key] = final
    #print("=====")

In [18]:
cnts = [str(len(x[0]))+','+str(len(x[1])) for x in filtered.values()]

In [19]:
Counter(cnts)

Counter({'14,0': 527, '0,0': 2640, '0,14': 408, '14,14': 63})

In [20]:
sum(Counter(cnts).values())

3638

In [21]:
#### Gene body length ####
forward = []
reverse = []
none = []
region_size = []
cnt = 0
for key, val in filtered.items():
    if len(val[1]) == 0 and len(val[0]) > 0: # forward direction
        #print("forward")
        x = val[0]
        start = x[4]
        end = x[5]
        dist = end - start
        bed = [x[0], start-dist, end, x[6], x[7], x[8], x[9], x[10], x[13], '.', '.', '.', '.', '.', '.']
        forward.append(bed)
    elif len(val[0]) == 0 and len(val[1]) > 0: # reverse direction
        #print("reverse")
        x = val[1]
        start = x[4]
        end = x[5]
        dist = end - start
        bed = [x[0], start, end + dist, '.', '.', '.', '.', '.', '.', x[6], x[7], x[8], x[9], x[10], x[13]]
        reverse.append(bed)
    elif len(val[0]) > 0 and len(val[1]) > 0:  # could be both 
        if val[0][10] > 2*val[1][10]: # forward
            #print("forward")
            x = val[0]
            start = x[4]
            end = x[5]
            dist = end - start
            bed = [x[0], start-dist, end, x[6], x[7], x[8], x[9], x[10], x[13], '.', '.', '.', '.', '.', '.']
            forward.append(bed)
        elif val[1][10] > 2*val[0][10]: # reverse
            #print("reverse")
            x = val[1]
            start = x[4]
            end = x[5]
            dist = end - start
            bed = [x[0], start, end + dist, '.', '.', '.', '.', '.', '.', x[6], x[7], x[8], x[9], x[10], x[13]]
            reverse.append(bed)
        else:
            cnt += 1
    else: # none
        chrom = key.split(':')[0]
        start = int(key.split(':')[1].split("-")[0])
        end = int(key.split(':')[1].split("-")[1])
        mid = int((start+end)/2)
        bed = [chrom, mid-150000, mid+150000, '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
        none.append(bed)
    region_size.append(bed[2]-bed[1])

In [22]:
print("forward : " + str(len(forward)))
print("reverse : " + str(len(reverse)))
print("none : " + str(len(none)))

forward : 556
reverse : 423
none : 2640


In [43]:
len(forward) + len(reverse) + len(none) + cnt

3638

In [47]:
write_result(directory, forward, 'RNAPII-peaks-loading_TSS-forward_genebodylen_20200711.bed')
write_result(directory, reverse, 'RNAPII-peaks-loading_TSS-reverse_genebodylen_20200711.bed')
write_result(directory, none, 'RNAPII-peaks-loading_non-TSS_genebodylen_20200711.bed')